## 2020

In [2]:
import os 
import time
import glob
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# === ฟังก์ชันรอการดาวน์โหลดไฟล์ ===
def wait_for_download(path, timeout=60):
    start = time.time()
    while time.time() - start < timeout:
        files = glob.glob(os.path.join(path, "*.xls*"))
        if files:
            latest = max(files, key=os.path.getctime)
            if not latest.endswith(".crdownload"):
                return latest
        time.sleep(1)
    return None

# === ตั้งค่าโฟลเดอร์ดาวน์โหลด (โฟลเดอร์ 2020) ===
download_dir = os.path.abspath("2020")
os.makedirs(download_dir, exist_ok=True)

options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# URL หลัก
base_url = "https://www.eppo.go.th/epposite/index.php/th/petroleum/price/oil-price?orders[publishUp]=publishUp&issearch=1"

# === เงื่อนไขเลือกปั๊ม ===
target_stations = ["ปตท", "บางจาก", "เชลล์"]

# === ไฟล์ CSV output ===
output_file = os.path.join(download_dir, "oil_price_E10_2020.csv")
with open(output_file, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "fuel_type", "effective_date", "price"])
    writer.writeheader()

    # === ช่วงเวลา 01/01/2020 → 31/12/2020 ===
    start_date = datetime.strptime("01/01/2020", "%d/%m/%Y")
    end_date   = datetime.strptime("31/12/2020", "%d/%m/%Y")

    for d in range((end_date - start_date).days + 1):
        date = (start_date + timedelta(days=d)).strftime("%d/%m/%Y")
        print(f"📌 Processing date: {date}")

        driver.get(base_url)

        # === ตรวจสอบ iframe และหา TbxToDate ===
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        date_box = None
        for i, iframe in enumerate(iframes):
            driver.switch_to.frame(iframe)
            try:
                date_box = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "TbxToDate"))
                )
                print(f"✅ เจอ TbxToDate ใน iframe {i}")
                break
            except:
                driver.switch_to.default_content()

        if not date_box:
            print("❌ ไม่เจอช่องใส่วันที่")
            continue

        # ✅ กรอกวันที่
        date_box.clear()
        date_box.send_keys(date)

        # ✅ กดปุ่ม Generate
        btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "BtnGenerate"))
        )
        driver.execute_script("arguments[0].click();", btn)

        driver.switch_to.default_content()

        # === รอไฟล์ดาวน์โหลด ===
        file_path = wait_for_download(download_dir, timeout=60)
        if not file_path:
            print(f"❌ ไม่มีไฟล์ดาวน์โหลดของ {date}")
            continue
        print(f"✅ เจอไฟล์ดาวน์โหลด: {file_path}")

        # === 1) แปลง Excel → CSV ===
        if file_path.endswith(".xls"):
            df = pd.read_excel(file_path, engine="xlrd", header=None)
        else:
            df = pd.read_excel(file_path, engine="openpyxl", header=None)

        temp_csv = file_path.replace(".xls", ".csv").replace(".xlsx", ".csv")
        df.to_csv(temp_csv, index=False, encoding="utf-8-sig")
        print(f"📂 แปลง Excel → CSV: {temp_csv}")

        # === 2) อ่าน CSV ใหม่ และ drop NaN ===
        csv_df = pd.read_csv(temp_csv, header=None)
        csv_df = csv_df.dropna(how="all").reset_index(drop=True)

        # === หา header row (ต้องมี PTT/ปตท) ===
        header_row = csv_df[csv_df.astype(str).apply(
            lambda r: any(x in str(r.values) for x in ["PTT", "ปตท"]), axis=1
        )]
        if header_row.empty:
            print("⚠️ ไม่พบ header row")
            continue
        stations = [str(x).strip() for x in header_row.iloc[0, 1:]]

        # === หา fuel row ของ Gasohol 95-E10 ===
        fuel_row = csv_df[csv_df.astype(str).apply(
            lambda r: "95" in str(r[0]) and "E10" in str(r[0]), axis=1
        )]
        if fuel_row.empty:
            print(f"⚠️ ไม่พบ Gasohol 95 E10 ใน {temp_csv}")
            continue
        fuel_type = fuel_row.iloc[0, 0]
        prices = fuel_row.iloc[0, 1:].tolist()

        # === หา effective date row ===
        eff_row = csv_df[csv_df.astype(str).apply(
            lambda r: "Effective Date" in str(r[0]), axis=1
        )]
        eff_dates = list(eff_row.iloc[0, 1:]) if not eff_row.empty else [""] * len(stations)

        # === กรองเฉพาะปั๊มที่ต้องการ ===
        for st, pr, eff in zip(stations, prices, eff_dates):
            if pd.isna(pr) or pr == "-" or st == "nan":
                continue
            if st not in target_stations:
                continue

            writer.writerow({
                "date": datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d"),
                "station": st,
                "fuel_type": fuel_type,
                "effective_date": eff,
                "price": pr
            })

driver.quit()
print(f"🎉 Done! All results saved to {output_file}")


📌 Processing date: 01/01/2020
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200101.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200101.csv
📌 Processing date: 02/01/2020
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200101.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200101.csv
📌 Processing date: 03/01/2020
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200102.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200102.csv
📌 Processing date: 04/01/2020
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200103.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200103.csv
📌 Processing date: 05/01/2020
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2020\EPPO_RetailOilPrice_on_20200104.xls

In [3]:
import pandas as pd

# === อ่าน CSV เดิม ===
file_path = r"C:\zzz\oil\scrape\2020\oil_price_E10_2020.csv"
df = pd.read_csv(file_path)

# === ลบ NaN ออก ===
df = df.dropna()

# === จัดรูปแบบวันที่หลัก ===
df["date"] = pd.to_datetime(df["date"]).dt.date

# === effective_date ไม่ต้องแปลง datetime ตรง ๆ ===
# แค่เก็บเป็น string แล้ว strip เว้นวรรค
df["effective_date"] = df["effective_date"].astype(str).str.strip()

# === Pivot Table ===
df_pivot = df.pivot_table(
    index=["date", "effective_date"],
    columns="station",
    values="price",
    aggfunc="first"
).reset_index()

# === ตั้งชื่อคอลัมน์ใหม่ ===
df_pivot = df_pivot.rename(
    columns={
        "บางจาก": "price_Bangchak",
        "ปตท": "price_PTT",
        "เชลล์": "price_Shell"
    }
)

# === Save CSV ===
save_path = r"C:\zzz\oil\scrape\oil_price_E10_2020.csv"
df_pivot.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"บันทึกไฟล์เรียบร้อยแล้วที่: {save_path}")


บันทึกไฟล์เรียบร้อยแล้วที่: C:\zzz\oil\scrape\oil_price_E10_2020.csv


## 2021

In [1]:
import os 
import time
import glob
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# === ฟังก์ชันรอการดาวน์โหลดไฟล์ ===
def wait_for_download(path, timeout=60):
    start = time.time()
    while time.time() - start < timeout:
        files = glob.glob(os.path.join(path, "*.xls*"))
        if files:
            latest = max(files, key=os.path.getctime)
            if not latest.endswith(".crdownload"):
                return latest
        time.sleep(1)
    return None

# === ตั้งค่าโฟลเดอร์ดาวน์โหลด (โฟลเดอร์ 2021) ===
download_dir = os.path.abspath("2021")
os.makedirs(download_dir, exist_ok=True)

options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# URL หลัก
base_url = "https://www.eppo.go.th/epposite/index.php/th/petroleum/price/oil-price?orders[publishUp]=publishUp&issearch=1"

# === เงื่อนไขเลือกปั๊ม ===
target_stations = ["ปตท", "บางจาก", "เชลล์"]

# === ไฟล์ CSV output ===
output_file = os.path.join(download_dir, "oil_price_E10_2021.csv")
with open(output_file, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "fuel_type", "effective_date", "price"])
    writer.writeheader()

    # === ช่วงเวลา 01/01/2021 → 31/12/2021 ===
    start_date = datetime.strptime("01/01/2021", "%d/%m/%Y")
    end_date   = datetime.strptime("31/12/2021", "%d/%m/%Y")

    for d in range((end_date - start_date).days + 1):
        date = (start_date + timedelta(days=d)).strftime("%d/%m/%Y")
        print(f"📌 Processing date: {date}")

        driver.get(base_url)

        # === ตรวจสอบ iframe และหา TbxToDate ===
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        date_box = None
        for i, iframe in enumerate(iframes):
            driver.switch_to.frame(iframe)
            try:
                date_box = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "TbxToDate"))
                )
                print(f"✅ เจอ TbxToDate ใน iframe {i}")
                break
            except:
                driver.switch_to.default_content()

        if not date_box:
            print("❌ ไม่เจอช่องใส่วันที่")
            continue

        # ✅ กรอกวันที่
        date_box.clear()
        date_box.send_keys(date)

        # ✅ กดปุ่ม Generate
        btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "BtnGenerate"))
        )
        driver.execute_script("arguments[0].click();", btn)

        driver.switch_to.default_content()

        # === รอไฟล์ดาวน์โหลด ===
        file_path = wait_for_download(download_dir, timeout=60)
        if not file_path:
            print(f"❌ ไม่มีไฟล์ดาวน์โหลดของ {date}")
            continue
        print(f"✅ เจอไฟล์ดาวน์โหลด: {file_path}")

        # === 1) แปลง Excel → CSV ===
        if file_path.endswith(".xls"):
            df = pd.read_excel(file_path, engine="xlrd", header=None)
        else:
            df = pd.read_excel(file_path, engine="openpyxl", header=None)

        temp_csv = file_path.replace(".xls", ".csv").replace(".xlsx", ".csv")
        df.to_csv(temp_csv, index=False, encoding="utf-8-sig")
        print(f"📂 แปลง Excel → CSV: {temp_csv}")

        # === 2) อ่าน CSV ใหม่ และ drop NaN ===
        csv_df = pd.read_csv(temp_csv, header=None)
        csv_df = csv_df.dropna(how="all").reset_index(drop=True)

        # === หา header row (ต้องมี PTT/ปตท) ===
        header_row = csv_df[csv_df.astype(str).apply(
            lambda r: any(x in str(r.values) for x in ["PTT", "ปตท"]), axis=1
        )]
        if header_row.empty:
            print("⚠️ ไม่พบ header row")
            continue
        stations = [str(x).strip() for x in header_row.iloc[0, 1:]]

        # === หา fuel row ของ Gasohol 95-E10 ===
        fuel_row = csv_df[csv_df.astype(str).apply(
            lambda r: "95" in str(r[0]) and "E10" in str(r[0]), axis=1
        )]
        if fuel_row.empty:
            print(f"⚠️ ไม่พบ Gasohol 95 E10 ใน {temp_csv}")
            continue
        fuel_type = fuel_row.iloc[0, 0]
        prices = fuel_row.iloc[0, 1:].tolist()

        # === หา effective date row ===
        eff_row = csv_df[csv_df.astype(str).apply(
            lambda r: "Effective Date" in str(r[0]), axis=1
        )]
        eff_dates = list(eff_row.iloc[0, 1:]) if not eff_row.empty else [""] * len(stations)

        # === กรองเฉพาะปั๊มที่ต้องการ ===
        for st, pr, eff in zip(stations, prices, eff_dates):
            if pd.isna(pr) or pr == "-" or st == "nan":
                continue
            if st not in target_stations:
                continue

            writer.writerow({
                "date": datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d"),
                "station": st,
                "fuel_type": fuel_type,
                "effective_date": eff,
                "price": pr
            })

driver.quit()
print(f"🎉 Done! All results saved to {output_file}")


📌 Processing date: 01/01/2021
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210101.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210101.csv
📌 Processing date: 02/01/2021
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210101.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210101.csv
📌 Processing date: 03/01/2021
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210102.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210102.csv
📌 Processing date: 04/01/2021
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210103.xls
📂 แปลง Excel → CSV: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210103.csv
📌 Processing date: 05/01/2021
✅ เจอ TbxToDate ใน iframe 1
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\oil\scrape\2021\EPPO_RetailOilPrice_on_20210104.xls

In [4]:
import pandas as pd

# === อ่าน CSV เดิม ===
file_path = r"C:\zzz\oil\scrape\2021\oil_price_E10_2021.csv"
df = pd.read_csv(file_path)

# === ลบ NaN ออก ===
df = df.dropna()

# === จัดรูปแบบวันที่หลัก ===
df["date"] = pd.to_datetime(df["date"]).dt.date

# === effective_date ไม่ต้องแปลง datetime ตรง ๆ ===
# แค่เก็บเป็น string แล้ว strip เว้นวรรค
df["effective_date"] = df["effective_date"].astype(str).str.strip()

# === Pivot Table ===
df_pivot = df.pivot_table(
    index=["date", "effective_date"],
    columns="station",
    values="price",
    aggfunc="first"
).reset_index()

# === ตั้งชื่อคอลัมน์ใหม่ ===
df_pivot = df_pivot.rename(
    columns={
        "บางจาก": "price_Bangchak",
        "ปตท": "price_PTT",
        "เชลล์": "price_Shell"
    }
)

# === Save CSV ===
save_path = r"C:\zzz\oil\scrape\oil_price_E10_2021.csv"
df_pivot.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"บันทึกไฟล์เรียบร้อยแล้วที่: {save_path}")


บันทึกไฟล์เรียบร้อยแล้วที่: C:\zzz\oil\scrape\oil_price_E10_2021.csv
